In [1]:
import mxnet as mx
import numpy as np
import cv2
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)

In [2]:
def get_image(filename):
    img = cv2.imread(filename)  # read image in b,g,r order
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # change to r,g,b order
    img = cv2.resize(img, (224, 224))  # resize to 224*224 to fit model
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)  # change to (channel, height, width)
    img = img[np.newaxis, :]  # extend to (example, channel, heigth, width)
    return img

In [3]:
all_layers = sym.get_internals()
all_layers.list_outputs()[-10:-1]

['bn1_moving_var',
 'bn1_output',
 'relu1_output',
 'pool1_output',
 'flatten0_output',
 'fc1_weight',
 'fc1_bias',
 'fc1_output',
 'softmax_label']

In [4]:
all_layers = sym.get_internals()
sym = all_layers['flatten0_output']
mod = mx.mod.Module(symbol=sym, context=mx.cpu())
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
mod.set_params(arg_params, aux_params)

In [5]:
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [6]:
import glob
positive_files = glob.glob("positive1/*")
negative_files = glob.glob("negative/*")
test_files = glob.glob("test1/*")

In [7]:
positive = []
negative = []

In [8]:
for each in positive_files:
    img = get_image(each)
    mod.forward(Batch([mx.nd.array(img)]))
    out = mod.get_outputs()[0].asnumpy()
    positive.append(out[0])

In [9]:
for each in negative_files:
    img = get_image(each)
    mod.forward(Batch([mx.nd.array(img)]))
    out = mod.get_outputs()[0].asnumpy()
    negative.append(out[0])

In [10]:
test = {}
for each in test_files:
    img = get_image(each)
    mod.forward(Batch([mx.nd.array(img)]))
    out = mod.get_outputs()[0].asnumpy()
    test[each] = out[0]

In [11]:
label = np.empty(len(positive))
label.fill(1)

In [12]:
label1 = np.empty(len(negative))
label1.fill(0)
labels = np.append(label, label1)

In [13]:
positive = np.array(positive)
negative = np.array(negative)
train = np.vstack([positive,negative])

In [14]:
from sklearn import svm
SVM = svm.SVC()
SVM.fit(train, labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [15]:
import itertools
for key, val in test.iteritems():
    print key +" : "+ str(SVM.predict(val))

test1/6.jpg : [ 0.]
test1/8.jpg : [ 0.]
test1/IMG_20170115_194714278.jpg : [ 1.]
test1/IMG_20170115_194549852.jpg : [ 1.]
test1/324.jpg : [ 0.]
test1/322.jpg : [ 0.]
test1/IMG_20170115_194640647.jpg : [ 1.]
test1/7.JPG : [ 0.]
test1/IMG_20170115_194538550.jpg : [ 1.]
test1/IMG_20170115_175041810_BURST000_COVER_TOP.jpg : [ 1.]
test1/66.jpg : [ 0.]
test1/428.jpg : [ 0.]
test1/311.jpg : [ 0.]
test1/2.jpg : [ 0.]
test1/IMG_20170115_194339643.jpg : [ 1.]
test1/IMG_20170115_194601388.jpg : [ 1.]
test1/338.jpg : [ 0.]
test1/1.jpg : [ 0.]
test1/3.png : [ 0.]
test1/4.jpg : [ 0.]
test1/IMG_20170115_194324252.jpg : [ 1.]
test1/IMG_20170115_194650105.jpg : [ 1.]
test1/337.jpg : [ 0.]
test1/IMG_20170115_194701666.jpg : [ 1.]
test1/IMG_20170115_194329614.jpg : [ 1.]
test1/IMG_20170115_194656841.jpg : [ 1.]
test1/96.jpg : [ 0.]
test1/IMG_20170115_194458775.jpg : [ 1.]
test1/379.jpg : [ 0.]
test1/5.jpg : [ 0.]
test1/IMG_20170115_194436833.jpg : [ 1.]
test1/335.jpg : [ 0.]
test1/IMG_20170115_194541843.

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local